In [1]:
from google.colab import drive
drive.mount('/content/drive')

import os
os.chdir("/content/drive/MyDrive/Colab Notebooks")

Mounted at /content/drive


In [9]:
!pip install -q transformers
import transformers 
from transformers import GPT2Model, GPT2Tokenizer 
from transformers import BartModel, BartTokenizer 
from transformers import BertModel, BertTokenizer 

import torch
import numpy as np
import matplotlib.pyplot as plt
from torch import nn
import torch.nn.utils.prune as prune

import copy
import json
from itertools import chain

# Models

* GPT2-XL 
* ~Pangu $\alpha$ 2.6B: [Intro from Huawei](https://www.huaweicloud.com/product/modelarts/pangu.html)~
* ~erlangshen 1.3B: [Intro from IDEA](https://huggingface.co/IDEA-CCNL/Erlangshen-MegatronBert-1.3B)~

structure of Chinese language models are really weird.  They're not well supported by huggingface. Too many bugs. Instead, we use 
* bart-large
* bert-large 

# Visualization & Sparsification


In [10]:
def get_params_plot(model, model_name, sparsity=0):
  # get params dist.
  fig, ax = plt.subplots(figsize=(12,8))
  total_param = torch.cat([params.flatten() for params in model.parameters()]).detach().numpy()
  hist, bins = np.histogram(total_param, bins = np.linspace(-1,1,201)) 
  ax.plot(bins[:-1],hist) 
  plt.xlabel("Weight values")
  plt.ylabel("Log Count")
  plt.yscale('log')
  plt.title("distribution of all weights(model:{}, sparsity:{}%)".format(model_name, sparsity*100), fontsize=25)
  plt.savefig('{}/figs/vis_all_params_{}_sparsity_{}%.png'.format(os.getcwd(), model_name, sparsity*100))
  
  # get params dist. in each layer
  for elem in model.named_children():
    if elem[0] in ['h', 'encoder', 'decoder']: 
      fig, ax = plt.subplots(figsize=(12,8))
      for idx, child in enumerate(elem[1].named_children()):
        layer_param = torch.cat([params.flatten() for params in child[1].parameters()]).detach().numpy()
        hist, bins = np.histogram(layer_param, bins = np.linspace(-1,1,201)) 
        ax.plot(bins[:-1],hist,label="{} hidden layer".format(idx)) 
      plt.xlabel("Weight values")
      plt.ylabel("Log Count")
      plt.yscale('log')
      plt.legend(loc="upper left", bbox_to_anchor=(1, 1))
      plt.title("distribution of weights across layers(model:{}, sparsity:{}%)".format(model_name, sparsity*100), fontsize=25)
      plt.savefig('{}/figs/vis_layers_params_{}_sparsity_{}%.png'.format(os.getcwd(), model_name, sparsity*100))

In [11]:
def prune_model(model, model_name, prune_portion):
  '''
  using global pruning to prune gpt2, bart, and bert. 
  '''
  def print_sparsity(module_lst, prune_flag):

    zero_params_count = sum([torch.sum(module[0].weight == 0) for module in module_lst])
    total_params_count = sum([module[0].weight.nelement() for module in module_lst])

    print(
        "Global sparsity of model: {} in hidden layers {} pruning: {:.2f}%".format(model_name, prune_flag,
                                                    100. * float(zero_params_count)
                                                        / float(total_params_count)
                                                    )
        )
    return None
    
  if model_name == 'BART-large':
    modules = chain(model.encoder.modules(), model.decoder.modules())
  elif model_name == 'BERT-large':
    modules = model.encoder.modules()
  else:
    modules = model.h.modules()

  module_lst = []
  for module in modules:
    if hasattr(module, 'weight'):
      module_lst.append((module, 'weight'))

  print_sparsity(module_lst, prune_flag = 'before')
  prune.global_unstructured(module_lst, 
                            pruning_method=prune.L1Unstructured, 
                            amount=prune_portion)
  for module, attr in module_lst: 
    prune.remove(module, attr)
  print_sparsity(module_lst, prune_flag = 'after')
  return model

In [12]:
# Method from https://discuss.pytorch.org/t/finding-model-size/130275/5
def get_model_size(model):
  param_size = 0
  for param in model.parameters():
      param_size += param.nelement() * param.element_size()
  buffer_size = 0
  for buffer in model.buffers():
      buffer_size += buffer.nelement() * buffer.element_size()

  size_all_mb = (param_size + buffer_size) / 1024**2
  print('model size: {:.3f}MB'.format(size_all_mb))
  return size_all_mb

In [13]:
model_names = ['GPT2-xl', 'BART-large', 'BERT-large']
sparsity = [0, 0.1, 0.5, 0.9, 0.95, 0.99] 
model_size_dic = dict()

print("Start sparsifying...")
for model_name in model_names:
  if model_name == 'GPT2-xl':
    # model = GPT2Model.from_pretrained('gpt2-xl')
    model = GPT2Model.from_pretrained('gpt2')
  elif model_name == 'BART-large':
    # model = BartModel.from_pretrained("facebook/bart-large")
    model = BartModel.from_pretrained("facebook/bart-base")
  else:
    model = BertModel.from_pretrained("bert-base-cased")
    # model = BertModel.from_pretrained("bert-large-cased")

  for prune_proportion in sparsity:
    target_model = copy.deepcopy(model)
    if prune_proportion != 0:
      target_model = prune_model(target_model, model_name, prune_proportion)
    model_size_dic[f'{model_name}_sparsity_{prune_proportion*100}%'] = get_model_size(target_model)
    get_params_plot(target_model, model_name, prune_proportion)
    target_model.save_pretrained("{}/models/{}_sparsity_{}".format(os.getcwd(), 
                                                            model_name,
                                                            prune_proportion*100,
                                                            ))
    print("model {} with sparsity {} saved".format(model_name, prune_proportion * 100))
    
print("Sparsification completed!")


Output hidden; open in https://colab.research.google.com to view.

In [14]:
dir_open = open(f'{os.getcwd()}/model_size.json', "w")
json.dump(model_size_dic, dir_open, indent = 4)

# Evaluation on Benchmarks

GLUE(cola) and CLM(using wikitext)

~For pangu, erlangshen: CLUE (tnews and cmnli)~

In [2]:
!pip install -q transformers
import transformers 
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AutoModelForCausalLM 
from transformers import TrainingArguments, Trainer, DataCollatorWithPadding

import torch
import numpy as np
import pandas as pd 

!pip install -q datasets
import datasets
from datasets import load_metric, load_dataset

import json

     |████████████████████████████████| 5.5 MB 4.7 MB/s 
     |████████████████████████████████| 7.6 MB 50.1 MB/s 
     |████████████████████████████████| 163 kB 83.8 MB/s 
     |████████████████████████████████| 441 kB 4.9 MB/s 
     |████████████████████████████████| 115 kB 66.8 MB/s 
     |████████████████████████████████| 212 kB 84.6 MB/s 
     |████████████████████████████████| 95 kB 5.8 MB/s 
     |████████████████████████████████| 127 kB 85.6 MB/s 
     |████████████████████████████████| 115 kB 87.7 MB/s 


In [3]:
import os 
# change dir
os.chdir("/content/drive/MyDrive/Colab Notebooks")

In [4]:
model_names = ['GPT2-xl', 'BART-large', 'BERT-large']
tokenizers_address = ['gpt2', 'facebook/bart-base', "bert-base-cased"]
# tokenizers_address = ['gpt2-xl', 'facebook/bart-large', "bert-large-cased"]
# sparsity = [0, 0.1, 0.5, 0.9, 0.95, 0.99] 
sparsity = [0.5] 
models = dict()
tokenizers = dict()
for model_name, tokenizer_add in zip(model_names, tokenizers_address):
  tokenizers[model_name] = tokenizer_add
  models[model_name] = []
  for prune_proportion in sparsity:
    models[model_name].append(f'{os.getcwd()}/models/{model_name}_sparsity_{prune_proportion*100}')

## Running Benchmarks

In [5]:
def run_benchmarks(models, sparsity, tokenizers, model_names):
  '''
  run benchmarks
    GLUE(cola) and CLM(wikitext v2 raw)
  '''
  # from https://ifwind.github.io/2021/08/31/BERT实战——（7）生成任务-语言模型/#调用分词器对所有的文本分词
  # for  CLM
  def group_texts(examples):
    block_size = 256
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    total_length = (total_length // block_size) * block_size
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result
  
  # CLM
  def tokenize_function_clm(examples):
    return tokenizer(examples["text"], truncation=True)

  # GLUE
  def tokenize_function_glue(examples):
    return tokenizer(examples["sentence"], truncation=True)
  
  # for GLUE 
  def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

  for model_name in model_names:
    tokenizer = AutoTokenizer.from_pretrained(tokenizers[model_name])
    tokenizer.pad_token = tokenizer.eos_token
    data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
    for idx, model in enumerate(models[model_name]):
      for task in ['clm', 'glue']:
        if task == 'clm':
          dataset = datasets.load_dataset('wikitext', 'wikitext-2-raw-v1')
          tokenized_datasets = dataset.map(tokenize_function_clm, batched=True, remove_columns=["text"])
          tokenized_datasets = tokenized_datasets.map(group_texts, batched=True, batch_size=1000)
          model = AutoModelForCausalLM.from_pretrained(models[model_name][idx])
        else:
          dataset = datasets.load_dataset('glue', 'cola')
          metric = load_metric("glue", "cola")
          tokenized_datasets = dataset.map(tokenize_function_glue, batched=True)
          model = AutoModelForSequenceClassification.from_pretrained(models[model_name][idx], 
                                                                    num_labels=2)
        
        model.config.pad_token_id = model.config.eos_token_id
        train_dataset = tokenized_datasets["train"].shuffle(seed=42)
        eval_dataset = tokenized_datasets["validation"].shuffle(seed=42)
        training_args = TrainingArguments(output_dir=f"{os.getcwd()}/training/{model_name}_{task}_sparsity_{sparsity[idx]*100}_output", 
                                    num_train_epochs=1,
                                    per_device_train_batch_size=32,
                                    per_device_eval_batch_size=32,
                                    save_strategy = "steps",
                                    save_steps = 10000,
                                    evaluation_strategy="epoch")
        
        if task == 'clm':
          trainer = Trainer(
                            model=model,
                            args=training_args,
                            train_dataset=train_dataset,
                            eval_dataset=eval_dataset,
                            data_collator=data_collator,
                            )
        else:
          trainer = Trainer(
                            model=model,
                            args=training_args,
                            train_dataset=train_dataset,
                            eval_dataset=eval_dataset,
                            data_collator=data_collator,
                            compute_metrics=compute_metrics,
                            )

        train_results = trainer.train()
        eval_results = trainer.evaluate()
        result = {'train': train_results.metrics,
                  'eval': eval_results}
        dir_open = open(f'{os.getcwd()}/benchmark/{model_name}_{task}_sparsity_{sparsity[idx]*100}_results.json', "w")
        json.dump(result, dir_open, indent = 4)

In [6]:
run_benchmarks(models, sparsity, tokenizers, model_names)

Downloading:   0%|          | 0.00/665 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Generating test split:   0%|          | 0/4358 [00:00<?, ? examples/s]

...

  0%|          | 0/9 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

loading configuration file /content/drive/MyDrive/Colab Notebooks/models/BART-large_sparsity_50.0/config.json
Model config BartConfig {
  "_name_or_path": "/content/drive/MyDrive/Colab Notebooks/models/BART-large_sparsity_50.0",
  "activation_dropout": 0.1,
  "activation_function": "gelu",
  "add_bias_logits": false,
  "add_final_layer_norm": false,
  "architectures": [
    "BartModel"
  ],
  "attention_dropout": 0.1,
  "bos_token_id": 0,
  "classif_dropout": 0.1,
  "classifier_dropout": 0.0,
  "d_model": 768,
  "decoder_attention_heads": 12,
  "decoder_ffn_dim": 3072,
  "decoder_layerdrop": 0.0,
  "decoder_layers": 6,
  "decoder_start_token_id": 2,
  "dropout": 0.1,
  "early_stopping": true,
  "encoder_attention_heads": 12,
  "encoder_ffn_dim": 3072,
  "encoder_layerdrop": 0.0,
  "encoder_layers": 6,
  "eos_token_id": 2,
  "forced_bos_token_id": 0,
  "forced_eos_token_id": 2,
  "gradient_checkpointing": false,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1"
  },
  "init_std": 0

ValueError: ignored

In [ ]:
!nvidia-smi

Mon Nov 14 03:51:35 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  A100-SXM4-40GB      Off  | 00000000:00:04.0 Off |                    0 |
| N/A   32C    P0    48W / 400W |  40364MiB / 40536MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!fuser -v /dev/nvidia*

                     USER        PID ACCESS COMMAND
/dev/nvidia0:        root       1092 F...m python3
/dev/nvidiactl:      root       1092 F...m python3
/dev/nvidia-uvm:     root       1092 F...m python3
